In [31]:
import sqlite3
import pandas as pd
import time

st = time.time()

conn = sqlite3.connect("stock.db")
ds_price = pd.read_sql_query('select STOCK_CODE, STOCK_DATE, STOCK_CLOSE from VCSC_STOCK_PRICE', conn)
conn.close()

print("1--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['STOCK_DATE'] = pd.to_datetime(ds_price['STOCK_DATE'], format='%m/%d/%Y')
ds_price['STOCK_YEAR'] = ds_price['STOCK_DATE'].dt.year
ds_price['STOCK_QUARTER'] = ds_price['STOCK_YEAR'].astype('str') + ds_price['STOCK_DATE'].dt.quarter.astype('str')
ds_price['STOCK_CLOSE'] = ds_price['STOCK_CLOSE'].str.replace(',', '').astype(float)

ds_price = ds_price.sort_values(['STOCK_DATE'], ascending=[False]).groupby(['STOCK_CODE', 'STOCK_YEAR']).nth(0)
ds_price['NEXT-PRICE-Y'] = ds_price.groupby(level=0)['STOCK_CLOSE'].shift(-1)

print("2--- %s seconds ---" % (time.time() - st))
st = time.time()

ds_price['RETURN'] = ds_price['NEXT-PRICE-Y'].div(ds_price['STOCK_CLOSE'])
ds_price['DECILE'] = (ds_price.groupby('STOCK_YEAR')['STOCK_CLOSE']
                                .apply(lambda x: pd.qcut(x,10,duplicates='drop',labels=False)))
ds_price.reset_index(drop = False, inplace = True)
s5 = ds_price.groupby(['STOCK_YEAR','DECILE'], as_index=False).mean()

print("3--- %s seconds ---" % (time.time() - st))
st = time.time()

s5.pivot(index='DECILE', columns='STOCK_YEAR', values='RETURN')


1--- 1.5156962871551514 seconds ---
2--- 5.194401025772095 seconds ---
3--- 0.08876395225524902 seconds ---


STOCK_YEAR,2011,2012,2013,2014,2015,2016,2017,2018
DECILE,,,,,,,,
0,1.897128,1.815272,2.158168,1.462993,1.154056,1.299941,0.844294,NaN
1,1.528148,1.738864,1.643496,1.184147,1.215439,1.732884,0.950258,NaN
2,1.337873,1.459011,1.610471,1.218089,1.048417,1.277053,0.962215,NaN
3,1.274146,1.335617,1.438198,1.090829,1.213372,1.255752,0.963837,NaN
4,1.248606,1.312559,1.299000,1.078239,1.129736,1.168102,0.996121,NaN
5,1.191051,1.264219,1.279644,1.187228,1.075416,1.201508,1.055064,NaN
6,1.131183,1.412297,1.311826,1.107599,1.179065,1.185163,0.937170,NaN
7,1.080604,1.351868,1.328039,1.330390,1.203371,1.147565,0.959218,NaN
8,1.074095,1.241823,1.295672,1.110942,1.225896,1.020493,0.941860,NaN
